In [84]:
import pandas as pd
from Levenshtein import editops, distance
from collections import Counter

In [116]:
df = pd.read_csv('a_ToCheck.csv', encoding='utf-8', sep='\t', low_memory=False)

In [49]:
unique = df[df['new_ngram'].notna()].drop_duplicates(subset='new_ngram')

In [51]:
unique.columns

Index(['raw_ngram', 'year', 'match_count', 'volume_count', 'idx', 'n_gram',
       'new_idx', 'is_bastard', 'new_ngram'],
      dtype='object')

In [59]:
corrections = unique['new_ngram'].astype(str).tolist()
errors = unique['n_gram'].astype(str).tolist()

In [65]:
dists=[]
edit_ops=[]

In [76]:
for pair in zip(corrections, errors):
    print(pair)
    print(distance(pair[0],pair[1]))
    dists.append(distance(pair[0],pair[1]))

('Августа', 'Авгуота')
1
('Августъ', 'Августь')
1
('Австріи', 'Австрж')
2
('Агаѳангела', 'Агаоангела')
1
('Аѳанасьевской', 'Аеанасьевской')
1
('Аѳонасій', 'Аеонасш')
3
('Аѳонѣ', 'Аеонт')
2
('Australian', 'Аиз1гаНап')
10
('Акcаковъ', 'Акеаковъ')
1
('Алекcея', 'Алекеея')
1
('Алексѣй', 'АлексЪй')
1
('Александринскаго', 'Александрннскаго')
1
('Александровича', 'Александровича2')
1
('Александровскаго', 'Александровскаю')
2
('Алексѣя', 'Алексая')
1
('Альпійскую', 'Альшйскую')
2
('Амфилохія', 'Амфилохгя')
1
('Анастасія', 'Анасташ')
3
('Анатоліе', 'Анатолге')
1
('Андреевичъ', 'Андреевпчъ')
1
('Апостолы', 'Аностолы')
1
('Арестованіе', 'Арестоваше')
2
('Арзамасскій', 'Арзамасскш')
2
('Артиллерійскаго', 'АртиллерШскаго')
2
('Архангельскій', 'Архангельскш')
2
('Архіерейскаго', 'Архгерейскаго')
1
('Архимандритовъ', 'АрхимандритовЪ')
1
('Aceraceae', 'Асегасеае')
9
('Афанасій', 'Афанасш')
2
('дѣятелей', 'Ъятелей')
2
('Lazare', 'Ьагаге')
6
('Lady', 'Ьайу')
4
('banke', 'Ьапке')
5
('ЯЗЫК', 'Я8ЫК')
1
('Я

In [74]:
dist_counts = Counter(dists)

In [79]:
print('''
{} % of errors have edit distance 1\n
{} % of errors have edit distance 2\n
{} % of errors have edit distance 3\n
{} % of errors have edit distance more than 3 (probably it is transliteration)
'''.format(dist_counts[1]/len(dists)*100, 
           dist_counts[2]/len(dists)*100, 
           dist_counts[3]/len(dists)*100,
           (len(dists)-dist_counts[1]-dist_counts[2]-dist_counts[3])/len(dists)*100))


53.65853658536586 % of errors have edit distance 1

26.82926829268293 % of errors have edit distance 2

7.317073170731707 % of errors have edit distance 3

12.195121951219512 % of errors have edit distance more than 3 (probably it is transliteration)



In [113]:
error_df=''

In [114]:
for pair in zip(corrections, errors):
    if len(editops(pair[0],pair[1]))==1:
        editop, idxw, idxx = editops(pair[0],pair[1])[0]
        if editop == 'replace':
            correct_char=pair[0][idxw]
            error_char=pair[1][idxx]
            ew=pair[1][idxx]+'|'+pair[0][idxw]
        elif editop == 'insert':
            correct_char = '-'
            error_char = pair[1][idxx]
            ew=pair[1][idxx-1:idxx+1]+'|'+pair[0][idxw-1]
        elif editop == 'delete':
            correct_char = pair[0][idxw]
            error_char = '-'
            ew=pair[1][idxx-1]+'|'+pair[0][idxw-1:idxw+1]
        df_temp = pd.DataFrame([[pair[0],
                                  pair[1],
                                  editop,
                                  correct_char, 
                                 error_char, ew]],columns=['correction', 
                                                             'error', 
                                                             'editop', 
                                                             'correct_char', 
                                                             'error_char', 
                                                             'e|w'])
    if not len(error_df):
        error_df=df_temp
    else:
        error_df=pd.concat([error_df,df_temp],axis=0,ignore_index=True)

In [115]:
error_df

,correction,error,editop,correct_char,error_char,e|w
0,Августа,Авгуота,replace,с,о,о|с
1,Августъ,Августь,replace,ъ,ь,ь|ъ
2,Августъ,Августь,replace,ъ,ь,ь|ъ
3,Агаѳангела,Агаоангела,replace,ѳ,о,о|ѳ
4,Аѳанасьевской,Аеанасьевской,replace,ѳ,е,е|ѳ
5,Аѳанасьевской,Аеанасьевской,replace,ѳ,е,е|ѳ
6,Аѳанасьевской,Аеанасьевской,replace,ѳ,е,е|ѳ
7,Аѳанасьевской,Аеанасьевской,replace,ѳ,е,е|ѳ
8,Акcаковъ,Акеаковъ,replace,c,е,е|c
9,Алекcея,Алекеея,replace,c,е,е|c


<img src="pics/Снимок.PNG",width=600, height=300>
see https://web.stanford.edu/~jurafsky/slp3/B.pdf

In [120]:
sub_counts=Counter(error_df[error_df.editop=='replace']['e|w'].astype(str).tolist())
ins_counts=Counter(error_df[error_df.editop=='insert']['e|w'].astype(str).tolist())
char_counts=Counter([char if char!='-' else error_df['e|w'][i][-1] for i, char in enumerate(error_df.correct_char.astype(str).tolist())])

In [124]:
def P_ew(editop, ew):
    if editop=='replace':
        return sub_counts[ew]/char_counts[ew[-1]]
    elif editop=='insert':
        return ins_counts[ew]/char_counts[ew[-1]]
    else:
        return 0.0001 # костыль (нужно сглаживание)

In [132]:
P_ew('replace', 'ь|ъ')

0.2222222222222222